# **Installing packages and importing libraries**

In [1]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



# **Ex2: Part 1 to 6** - *Problem Fomulation, Model, and Solver without the approaches and also with Approach 1 and Approach 2.*



\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
-x_1 - x_2 & \leq -10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
-100000 x_1 - 500000 x_2 - 200000 x_3 - 45000 x_4 - 25000 x_5 & \leq -900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}
---

# ***Without using any approach.***

Our problem to be solved :


This is not a linear programming problem but in can be converted to one by a simple adjustement. Since the adjustment only needs to be done to take care of the non linear constraint, we can simply do that by using the definition of absolute function. |x| <= a can be written as -a <= x <= a.


 \begin{array}{l}
 | ( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) |\ \leqslant 18 \\ \\
and\ adjusting\ constraints\ \\
\\
( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) \ \leqslant 18\\
( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) \ \geqslant \ -18\\
\\
or\ \\
( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) \ \leqslant \ 18\\
( x_{3} +x_{4} +x_{5}) -( x_{1} +x_{2}) \ \leqslant \ 18
\end{array}



In [2]:
coef = np.array([
                 [35,57,48,20,15,0],
                 [1,1,1,0,0,75],
                 [0,0,1,1,1,100],
                 [-1,-1,0,0,0,-10],
                 [30000,95000,10000,5000,4000,1500000],
                 [-100000,-500000,-200000,-45000,-25000,-900000],
                 [1,1,-1,-1,-1,18],
                 [-1,-1,1,1,1,18],

                 ])

N = coef.shape[1]-1
M = coef.shape[0]-1

col_index = np.arange(N)
row_index = np.arange(M)

model = ConcreteModel()

model.x = Var(col_index, domain = NonNegativeIntegers)
model.obj = Objective(expr = summation(coef[0,:-1],model.x) , sense=maximize)
model.cons = ConstraintList()

for i in row_index:
  model.cons.add(summation(coef[i+1,:-1],model.x) <= coef[i+1,-1])
 

cbc = SolverFactory('cbc')
result = cbc.solve(model)


print('Status :',result.solver.status)
print('Termination Condition :',result.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model.obj())
print('\n')
print('The value of the variables are : ')
print('\n')
for i in range(N):
  print('[x',i,'] :', model.x[i].value)



Status : ok
Termination Condition : optimal
The optimal vlaue of the objective function is : 3378.0


The value of the variables are : 


[x 0 ] : 34.0
[x 1 ] : 0.0
[x 2 ] : 41.0
[x 3 ] : 11.0
[x 4 ] : 0.0


# ***Using Approach 1.***


To get aroung the problem of having a non linear constraint.

 \begin{array}{l}
 Let: \ \ \ \ \
y\ =\ \ | ( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) |\ =\ \ | x_{1} +x_{2} -x_{3} -x_{4} -x_{5} |\\
\\
Also,\ we\ will\ add\ the\ following\ constraints\\
\\
y\geqslant 0\ \ \ \ \  y\leqslant 18
\\
y\ \geqslant \ x_{1} +x_{2} -x_{3} -x_{4} -x_{5} \ \ \ \ \ \ \ \ or\ \ \ \ \ \ \ \ \ x_{1} +x_{2} -x_{3} -x_{4} -x_{5} -y\ \leqslant 0\\
y\ \geqslant \ -( x_{1} +x_{2} -x_{3} -x_{4} -x_{5}) \ \ \ \ \ \ \ \ \ \ or\ \ \ \ \ \ \ \ \ \ \ -x_{1} -x_{2} +x_{3} +x_{4} +x_{5} -y\ \leqslant 0
\end{array}


In [16]:
coef_A1 = np.array([
                 [35,57,48,20,15,0,0],
                 [1,1,1,0,0,0,75],
                 [0,0,1,1,1,0,100],
                 [-1,-1,0,0,0,0,-10],
                 [30000,95000,10000,5000,4000,0,1500000],
                 [-100000,-500000,-200000,-45000,-25000,0,-900000],
                 [1,1,-1,-1,-1,-1,0],
                 [-1,-1,1,1,1,-1,0],

                 ])


N_A1 = coef_A1.shape[1]-2
M_A1 = coef_A1.shape[0]-1

col_index_x = np.arange(N_A1)
row_index = np.arange(M_A1)

model_A1 = ConcreteModel()

model_A1.x = Var(col_index_x, domain = NonNegativeIntegers)
model_A1.y = Var(domain=NonNegativeIntegers)

model_A1.obj = Objective(expr = summation(coef_A1[0,:-2],model_A1.x) , sense=maximize)
model_A1.cons = ConstraintList()

for i in row_index:
  model_A1.cons.add(summation(coef_A1[i+1,:-2],model_A1.x) +(model_A1.y * coef_A1[i+1,-2]) <= coef_A1[i+1,-1])
 
model_A1.y.setub(18)




result_A1 = cbc.solve(model_A1)


print('Status :',result_A1.solver.status)
print('Termination Condition :',result_A1.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model_A1.obj())
print('\n')
print('The value of the variables are : ')
print('\n')
for i in range(N_A1):
  print('[x',i,'] :', model_A1.x[i].value)

print('[y] :', model_A1.y.value)
print('\n')
model_A1.cons.display()

Status : ok
Termination Condition : optimal
The optimal vlaue of the objective function is : 3378.0


The value of the variables are : 


[x 0 ] : 34.0
[x 1 ] : 0.0
[x 2 ] : 41.0
[x 3 ] : 11.0
[x 4 ] : 0.0
[y] : 18.0


cons : Size=7
    Key : Lower : Body        : Upper
      1 :  None :        75.0 :      75.0
      2 :  None :        52.0 :     100.0
      3 :  None :       -34.0 :     -10.0
      4 :  None :   1485000.0 : 1500000.0
      5 :  None : -12095000.0 : -900000.0
      6 :  None :       -36.0 :       0.0
      7 :  None :         0.0 :       0.0


**ACTIVE CONSTRAINTS : 1,7**

#  ***Using Approach 2.***


To get aroung the problem of having a non linear constraint.

 \begin{array}{l}
Let\ \\
\\
\ \ \ \ \ \ \ \ \ \ \ \ \ \ | ( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) |\ =\ a+b\\
and\ \ \ \ \ \ \ ( x_{1} +x_{2}) -( x_{3} +x_{4} +x_{5}) \ =\ a-b\\
\\
constaints\ additoinal\\
a+b\ \leqslant 18\\
a,b\geqslant 0\\
\end{array}



In [17]:
coef_A2 = np.array([
                 [35,57,48,20,15,0],
                 [1,1,1,0,0,75],
                 [0,0,1,1,1,100],
                 [-1,-1,0,0,0,-10],
                 [30000,95000,10000,5000,4000,1500000],
                 [-100000,-500000,-200000,-45000,-25000,-900000],

                 ])


N_A2 = coef_A2.shape[1]-1
M_A2 = coef_A2.shape[0]-1

col_index_x = np.arange(N_A2)
row_index = np.arange(M_A2)

model_A2 = ConcreteModel()

model_A2.x = Var(col_index_x, domain = NonNegativeIntegers)
model_A2.a = Var(domain=NonNegativeReals)
model_A2.b = Var(domain=NonNegativeReals)

model_A2.obj = Objective(expr = summation(coef_A2[0,:-1],model_A2.x) , sense=maximize)
model_A2.cons = ConstraintList()

for i in row_index:
  model_A2.cons.add(summation(coef_A2[i+1,:-1],model_A2.x) <= coef_A2[i+1,-1])
 
model_A2.cons.add(model_A2.x[0] + model_A2.x[1] - model_A2.x[2] - model_A2.x[3] - model_A2.x[4] - model_A2.a + model_A2.b == 0) 
model_A2.cons.add(model_A2.a + model_A2.b <= 18)




result_A2 = cbc.solve(model_A2)
print('Status :',result_A2.solver.status)
print('Termination Condition :',result_A2.solver.termination_condition)
print('The optimal vlaue of the objective function is :',model_A2.obj())
print('\n')
print('The value of the variables are : ')
print('\n')
for i in range(N_A2):
  print('[x',i,'] :', model_A2.x[i].value)

print('[a] :', model_A2.a.value)
print('[b] :', model_A2.b.value)

print('\n')

model_A2.cons.display()



Status : ok
Termination Condition : optimal
The optimal vlaue of the objective function is : 3378.0


The value of the variables are : 


[x 0 ] : 34.0
[x 1 ] : 0.0
[x 2 ] : 41.0
[x 3 ] : 11.0
[x 4 ] : 0.0
[a] : 0.0
[b] : 18.0


cons : Size=7
    Key : Lower : Body        : Upper
      1 :  None :        75.0 :      75.0
      2 :  None :        52.0 :     100.0
      3 :  None :       -34.0 :     -10.0
      4 :  None :   1485000.0 : 1500000.0
      5 :  None : -12095000.0 : -900000.0
      6 :   0.0 :         0.0 :       0.0
      7 :  None :        18.0 :      18.0


***Active Constraints : 1,6,7***

# **Ex2: Part 7** - *Adding the new constraint related to z3 in the model.*

We are given tha z3 = x4 + x5
And we want to add the constraints that |z3 - x3| >= 4 

This is not a linear progrma but can be converted to one. To solve the above problem with the the approaches we will make some changes to the formulation of the problem. 

**For Approach 1 :**

 \begin{array}{l}
Let\ \\
y_{2} \ =\ \ | x_{4} +x_{5} -x_{3} |\\
Also,\ we\ will\ add\ the\ following\ constraints\\
\\
y_{2} \geqslant 0\ and\ y_{2} \ \geqslant 4\\
y_{2} \ \geqslant x_{4} +x_{5} -x_{3} \ \ \ \ \ \ \ \ or\ \ \ \ \ \ \ \ \ x_{4} +x_{5} -x_{3} -y_{2} \ \leqslant 0\\
y_{2} \ \geqslant \ -( x_{4} +x_{5} -x_{3}) \ \ \ \ \ \ \ \ \ or\ \ \ \ \ \ \ \ \ \ \ -x_{4} -x_{5} +x_{3} -y_{2} \ \leqslant 0
\end{array}

**For Approach 2 :**

 \begin{array}{l}
Let\ \\
\\
\ \ \ \ \ \ \ \ \ \ \ \ \ \ | x_{4} +x_{5} -x_{3} |\ =\ a_{2} +b_{2}\\
and\ \ \ \ \ \ \ x_{4} +x_{5} -x_{3} =\ a_{2} -b_{2}\\
\\
constaints\ additoinal\\
a_{2} +b_{2} \ \geqslant 4\\
a_{2},b_{2}\geqslant 0
\end{array}

By making these changes to the problem we can solve them.
